In [ ]:
import pyautogui
import cv2
import numpy as np
import time
import os

def capture_screen(region=None):
    screenshot = pyautogui.screenshot(region=region)
    frame = np.array(screenshot)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    return frame

save_dir = "dataset"

if not os.path.exists(save_dir):
    os.makedirs(save_dir)

region = (897, 68, 1432, 304)  # 設定截圖區域

while True:
    frame = capture_screen(region)
    timestamp = int(time.time() * 1000)
    
    frame = cv2.resize(frame,(535,236))
    cv2.imshow('Game', frame)

    key = cv2.waitKey(1) & 0xFF
    if key == ord('q'):
        break
    elif key == ord('f'):  # 按 'f' 標記跳躍
        duration = 0
        cv2.imwrite(os.path.join(save_dir, f"{timestamp}_jump_{duration}.png"), frame)
    elif key == ord('j'):  # 按 'j' 標記下滑
        duration = 2
        cv2.imwrite(os.path.join(save_dir, f"{timestamp}_slide_{duration}.png"), frame)
    elif key == ord('l'):  # 按 'l' 標記不做任何事
        duration = 0
        cv2.imwrite(os.path.join(save_dir, f"{timestamp}_none_{duration}.png"), frame)

cv2.destroyAllWindows()


In [ ]:
import os
import cv2
import numpy as np

def load_data(data_dir):
    images = []
    labels = []
    durations = []
    for filename in os.listdir(data_dir):
        if filename.endswith(".png"):
            parts = filename.split('_')
            label = parts[-2]
            duration = float(parts[-1].split('.')[0]) if label in ['jump', 'slide'] else 0
            if label == 'jump':
                labels.append(0)
            elif label == 'slide':
                labels.append(1)
            elif label == 'none':
                labels.append(2)
            durations.append(duration)
            img = cv2.imread(os.path.join(data_dir, filename))
            if img is not None:
                img = cv2.resize(img, (64, 64))  # 調整圖像大小
                images.append(img)
            else:
                print(f"Failed to read {filename}")
    images = np.array(images)
    labels = np.array(labels)
    durations = np.array(durations)
    return images, labels, durations

data_dir = "dataset"
X, y, durations = load_data(data_dir)
X = X / 255.0  # 標準化圖像數據


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

y = to_categorical(y, num_classes=3)  # 將標籤轉為one-hot程式碼
X_train, X_test, y_train, y_test, durations_train, durations_test = train_test_split(X, y, durations, test_size=0.2, random_state=42)

input_img = Input(shape=(64, 64, 3))
x = Conv2D(32, (3, 3), activation='relu')(input_img)
x = MaxPooling2D((2, 2))(x)
x = Conv2D(64, (3, 3), activation='relu')(x)
x = MaxPooling2D((2, 2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)

action_output = Dense(3, activation='softmax', name='action_output')(x)  # 動作輸出
duration_output = Dense(1, activation='linear', name='duration_output')(x)  # 持續時間輸出

model = Model(inputs=input_img, outputs=[action_output, duration_output])
model.compile(optimizer='adam', loss={'action_output': 'categorical_crossentropy', 'duration_output': 'mse'}, metrics={'action_output': 'accuracy', 'duration_output': 'mse'})

model.fit(X_train, {'action_output': y_train, 'duration_output': durations_train}, epochs=10, validation_data=(X_test, {'action_output': y_test, 'duration_output': durations_test}))
model.save('cookie_run_model.h5')


In [ ]:
import pyautogui
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import time


def capture_screen(region=None):
    screenshot = pyautogui.screenshot(region=region)
    frame = np.array(screenshot)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    return frame

def control_character(action, duration):  # 初始持續時間為0
    if action == 0:
        pyautogui.press('f') #按下跳躍
    if action == 1:
        pyautogui.keyDown('j')  # 按下滑鍵
        time.sleep(abs(float(duration)))
        pyautogui.keyUp('j')  # 放開下滑鍵
    if action == 2:
        pass  # 不做任何事情
        
custom_objects = {'mse': 'mean_squared_error'}# 自定義mse字典
model = load_model('cookie_run_model.h5', custom_objects=custom_objects)
region = (897, 68, 1432, 304)  # 設定截圖區域

while True:
    frame = capture_screen(region)
    img = cv2.resize(frame, (64, 64))
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    
    action_pred, duration_pred = model.predict(img)
    action = np.argmax(action_pred)
    duration = duration_pred[0][0]  # 獲取持續時間
    
    control_character(action, str(duration))
    
    frame = cv2.resize(frame,(535,236))
    cv2.imshow('Game', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
